# crowler

> Wed Domain Crowler 

In [ ]:
#| default_exp crowler

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import requests
import re
import urllib.request
from bs4 import BeautifulSoup
from collections import deque
from html.parser import HTMLParser
from urllib.parse import urlparse, urljoin, quote
import os
from typing import List, Optional
from urllib.error import URLError, HTTPError
from collections import deque


In [ ]:
#| export
# Regex pattern to match a URL
HTTP_URL_PATTERN = r'^http[s]*://.+'

In [ ]:
#| export

class HyperlinkParser(HTMLParser):
    """ 
    A parser to extract hyperlinks from an HTML document.
    Inherits from HTMLParser.
    """

    def __init__(self):
        """ Initialize the parser with an empty list of hyperlinks. """
        super().__init__()
        self.hyperlinks = []

    def handle_starttag(self, tag, attrs):
        """ 
        Override the HTMLParser's handle_starttag method.
        If the tag is an anchor tag and it has an href attribute, 
        add the href attribute to the list of hyperlinks.
        """
        attrs = dict(attrs)
        if tag == "a" and "href" in attrs:
            self.hyperlinks.append(attrs["href"])


In [ ]:
#| export

def get_hyperlinks(url: str) -> List[str]:
    """ 
    Function to get the hyperlinks from a URL. 

    Args:
    url (str): The URL from which to extract hyperlinks.

    Returns:
    list[str]: A list of hyperlinks, or an empty list if the URL couldn't be opened or its content is not HTML.
    """

    try:
        # Open the URL and read the HTML
        with urllib.request.urlopen(url) as response:
            # If the response is not HTML, return an empty list
            if not response.info().get('Content-Type', '').startswith("text/html"):
                print(f"Content at {url} is not HTML.")
                return []

            # Decode the HTML
            html = response.read().decode('utf-8')

    except HTTPError as e:
        print(f"HTTP Error: {e.code} for url: {url}")
        return []
    except URLError as e:
        print(f"URL Error: {e.reason} for url: {url}")
        return []
    except Exception as e:
        print(f"Unexpected error: {str(e)}")
        return []

    # Create the HTML Parser and then Parse the HTML to get hyperlinks
    parser = HyperlinkParser()
    parser.feed(html)

    return parser.hyperlinks


In [ ]:
#| export

def get_domain_hyperlinks(local_domain: str, url: str) -> List[str]:
    """ 
    Function to get the hyperlinks from a URL that are within the same domain.

    Args:
    local_domain (str): The local domain to match.
    url (str): The URL from which to extract hyperlinks.

    Returns:
    list[str]: A list of hyperlinks within the same domain, or an empty list if the URL couldn't be opened or its content is not HTML.
    """
    
    clean_links = set()

    for link in get_hyperlinks(url):
        # Ignore anchors and mailto links
        if link.startswith("#") or link.startswith("mailto:"):
            continue

        # Create absolute URL if the link is relative
        abs_link = urljoin(url, link)

        # Parse the URL and check if the domain is the same
        url_obj = urlparse(abs_link)
        if url_obj.netloc == local_domain:
            clean_links.add(abs_link.rstrip('/'))

    # Return the list of hyperlinks that are within the same domain
    return list(clean_links)


In [ ]:
#| export

def crawl(url: str, path: Optional[str] = ".") -> None:
    """ 
    Function to crawl a website starting from a given URL and save text content of each page to a file.

    Args:
    url (str): The URL to start crawling from.
    path (str, optional): The path where to create a directory to store the text files. Defaults to the current directory.
    """

    # Parse the URL and get the domain
    local_domain = urlparse(url).netloc

    # Create a queue to store the URLs to crawl
    queue = deque([url])

    # Create a set to store the URLs that have already been seen (no duplicates)
    seen = set([url])

    # Create a directory to store the text files
    text_path = os.path.join(path, "text", local_domain)
    os.makedirs(text_path, exist_ok=True)

    # While the queue is not empty, continue crawling
    while queue:
        # Get the next URL from the queue
        url = queue.pop()
        print(f"Crawling: {url}")  # for debugging and to see the progress

        try:
            # Get the hyperlinks from the URL and add them to the queue
            for link in get_domain_hyperlinks(local_domain, url):
                if link not in seen:
                    queue.append(link)
                    seen.add(link)

            # Read the content to retrieve text
            get_request = requests.get(url)
            get_request.raise_for_status()  # will raise HTTPError for bad status codes

            # Get the text from the URL using BeautifulSoup
            soup = BeautifulSoup(get_request.text, "html.parser")

            # Get the text but remove the tags
            text = soup.get_text()

            # If the crawler gets to a page that requires JavaScript, it will stop the crawl
            if "You need to enable JavaScript to run this app." in text:
                print(f"Unable to parse page {url} due to JavaScript being required")
                continue

            # Save text from the url to a <url>.txt file
            filename = os.path.join(text_path, quote(url[8:], safe="") + ".txt")
            with open(filename, "w", encoding="UTF-8") as f:
                f.write(text)

        except requests.exceptions.RequestException as e:
            print(f"Request error for url: {url}. Error details: {str(e)}")

        except Exception as e:
            print(f"Unexpected error for url: {url}. Error details: {str(e)}")


In [ ]:
#| eval : false
crawl("https://openai.com/","/home/hd/GptQA")

Crawling: https://openai.com/
Crawling: https://openai.com/pricing
Crawling: https://openai.com/pricing#language-models
Crawling: https://openai.com/contact-sales
Crawling: https://openai.com/contact-sales#content
Crawling: https://openai.com/customer-stories/stripe
Crawling: https://openai.com/customer-stories?topics=language
Crawling: https://openai.com/customer-stories/yabble
Crawling: https://openai.com/customer-stories/yabble#content
Crawling: https://openai.com/customer-stories#content
Crawling: https://openai.com/customer-stories/government-of-iceland
Crawling: https://openai.com/customer-stories/government-of-iceland#content
Crawling: https://openai.com/customer-stories/inworld-ai
Crawling: https://openai.com/customer-stories/inworld-ai#content
Crawling: https://openai.com/customer-stories/be-my-eyes
Crawling: https://openai.com/customer-stories/be-my-eyes#content
Crawling: https://openai.com/customer-stories/duolingo
Crawling: https://openai.com/customer-stories/duolingo#conte

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()